[![Labellerr](https://storage.googleapis.com/labellerr-cdn/%200%20Labellerr%20template/notebook.webp)](https://www.labellerr.com)

# **Face De-identification using Computer Vision**

---

[![labellerr](https://img.shields.io/badge/Labellerr-BLOG-black.svg)](https://www.labellerr.com/blog)
[![Youtube](https://img.shields.io/badge/Labellerr-YouTube-b31b1b.svg)](https://www.youtube.com/@Labellerr)
[![Github](https://img.shields.io/badge/Labellerr-GitHub-green.svg)](https://github.com/Labellerr/Hands-On-Learning-in-Computer-Vision)

## **Overview**
This notebook implements a **privacy-preserving video processing pipeline** that automatically detects and anonymizes human faces in video content using the YuNet deep learning model and OpenCV.
## 🔧 Core Components
| Component | Description |
|-----------|-------------|
| **YuNet Model** | Lightweight face detection model from OpenCV Zoo (ONNX format) |
| **FaceDetector Class** | Wrapper providing easy-to-use interface for face detection |
| **De-identification Methods** | Multiple techniques to anonymize detected faces |
## 🛡️ De-identification Methods
1. **Gaussian Blur** - Applies a smooth blur effect over face regions
2. **Pixelation** - Creates a mosaic effect by downscaling and upscaling face regions
3. **Black Box** - Completely obscures faces with solid black rectangles


## **Download YuNet Face Detection Model**
Clone the YuNet face detection model from Hugging Face. This model will be used for detecting faces in video frames.

In [ ]:
# !git clone https://huggingface.co/opencv/face_detection_yunet

## **Import Required Libraries**

Import OpenCV, NumPy, and other necessary modules for video processing and face detection.

In [1]:
import cv2 as cv
import os
import numpy as np
from typing import List, Tuple, Optional, Union
from pathlib import Path

from face_detection_yunet.yunet import YuNet

## **Face Detector Class**

A wrapper class for the YuNet face detection model. Provides methods for:
- Detecting faces and returning bounding boxes, confidence scores, and landmarks
- Processing videos frame-by-frame
- Visualizing detections with bounding boxes

In [3]:
class FaceDetector:
    """
    Face detector wrapper for YuNet model.
    Provides easy-to-use interface for face detection on images and videos.
    """
    
    def __init__(
        self,
        model_path: str = r'face_detection_yunet\face_detection_yunet_2023mar.onnx',
        conf_threshold: float = 0.9,
        nms_threshold: float = 0.3,
        top_k: int = 5000,
        backend_id: int = cv.dnn.DNN_BACKEND_OPENCV,
        target_id: int = cv.dnn.DNN_TARGET_CPU
    ):
        """
        Initialize the face detector.
        
        Args:
            model_path: Path to the YuNet ONNX model file
            conf_threshold: Confidence threshold for face detection (0.0-1.0)
            nms_threshold: NMS threshold for suppressing overlapping boxes
            top_k: Maximum number of faces to detect before NMS
            backend_id: OpenCV DNN backend ID
            target_id: OpenCV DNN target ID
        """
        self.model = YuNet(
            modelPath=model_path,
            inputSize=[320, 320],
            confThreshold=conf_threshold,
            nmsThreshold=nms_threshold,
            topK=top_k,
            backendId=backend_id,
            targetId=target_id
        )
        
    def detect_faces(self, frame: np.ndarray) -> List[dict]:
        """
        Detect faces in a single frame/image.
        
        Args:
            frame: Input image as numpy array (BGR format)
            
        Returns:
            List of dictionaries, each containing:
                - 'bbox': [x, y, width, height] - bounding box coordinates
                - 'confidence': float - detection confidence score
                - 'landmarks': List of 5 (x, y) tuples for facial landmarks
                  (right_eye, left_eye, nose_tip, right_mouth, left_mouth)
        """
        h, w = frame.shape[:2]
        self.model.setInputSize([w, h])
        
        # Perform inference
        results = self.model.infer(frame)
        
        # Parse results into structured format
        faces = []
        for det in results:
            face_data = {
                'bbox': [int(det[0]), int(det[1]), int(det[2]), int(det[3])],  # [x, y, w, h]
                'confidence': float(det[-1]),
                'landmarks': [
                    (int(det[4]), int(det[5])),   # right eye
                    (int(det[6]), int(det[7])),   # left eye
                    (int(det[8]), int(det[9])),   # nose tip
                    (int(det[10]), int(det[11])), # right mouth corner
                    (int(det[12]), int(det[13]))  # left mouth corner
                ]
            }
            faces.append(face_data)
            
        return faces
    
    def get_bboxes(self, frame: np.ndarray) -> List[List[int]]:
        """
        Get only bounding boxes from face detection (simplified interface).
        
        Args:
            frame: Input image as numpy array (BGR format)
            
        Returns:
            List of bounding boxes, each as [x, y, width, height]
        """
        faces = self.detect_faces(frame)
        return [face['bbox'] for face in faces]
    
    def process_video(
        self,
        video_path: str,
        output_path: Optional[str] = None,
        visualize: bool = False,
        progress_callback: Optional[callable] = None
    ) -> List[List[dict]]:
        """
        Process a video file and detect faces in each frame.
        
        Args:
            video_path: Path to input video file
            output_path: Optional path to save output video with visualizations
            visualize: Whether to display processing in real-time
            progress_callback: Optional callback function(frame_num, total_frames, faces)
            
        Returns:
            List of face detections for each frame. Each element is a list of face dicts.
        """
        cap = cv.VideoCapture(video_path)
        if not cap.isOpened():
            raise ValueError(f"Could not open video file: {video_path}")
        
        # Get video properties
        fps = cap.get(cv.CAP_PROP_FPS)
        width = int(cap.get(cv.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv.CAP_PROP_FRAME_HEIGHT))
        total_frames = int(cap.get(cv.CAP_PROP_FRAME_COUNT))
        
        # Setup video writer if output path is provided
        writer = None
        if output_path:
            fourcc = cv.VideoWriter_fourcc(*'mp4v')
            writer = cv.VideoWriter(output_path, fourcc, fps, (width, height))
        
        all_detections = []
        frame_num = 0
        
        try:
            while True:
                ret, frame = cap.read()
                if not ret:
                    break
                
                # Detect faces
                faces = self.detect_faces(frame)
                all_detections.append(faces)
                
                # Call progress callback if provided
                if progress_callback:
                    progress_callback(frame_num, total_frames, faces)
                
                # Visualize if requested
                if visualize or writer:
                    vis_frame = self._visualize_detections(frame.copy(), faces)
                    
                    if writer:
                        writer.write(vis_frame)
                    
                    if visualize:
                        cv.namedWindow('Face Detection', cv.WINDOW_NORMAL | cv.WINDOW_KEEPRATIO)
                        cv.setWindowProperty('Face Detection', cv.WND_PROP_FULLSCREEN, cv.WINDOW_FULLSCREEN)
                        if cv.waitKey(1) & 0xFF == ord('q'):
                            break
                
                frame_num += 1
                
        finally:
            cap.release()
            if writer:
                writer.release()
            if visualize:
                cv.destroyAllWindows()
        
        return all_detections
    
    def _visualize_detections(
        self,
        frame: np.ndarray,
        faces: List[dict],
        box_color: Tuple[int, int, int] = (0, 255, 0),
        text_color: Tuple[int, int, int] = (0, 0, 255),
        conf_label: bool = False
    ) -> np.ndarray:
        """
        Draw bounding boxes and landmarks on frame.
        
        Args:
            frame: Input frame to draw on
            faces: List of face detection dictionaries
            box_color: BGR color for bounding boxes
            text_color: BGR color for text
            conf_label: Whether to display confidence scores
            
        Returns:
            Frame with visualizations drawn
        """
        
        for face in faces:
            # Draw bounding box
            x, y, w, h = face['bbox']
            cv.rectangle(frame, (x, y), (x + w, y + h), box_color, 2)
            
            if conf_label:
                # Draw confidence score
                conf_text = f"{face['confidence']:.2f}"
                cv.putText(frame, conf_text, (x, y - 5), 
                          cv.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 2)
        
        return frame

## **Video Processing Demo**

Test the face detector on a sample video and visualize the detections.

In [4]:
detector = FaceDetector(conf_threshold=0.7)

sample_video_1 = r"video\sample_video_3.mp4"

detector.process_video(video_path=sample_video_1, 
                        output_path='sample_video_3_inference.mp4',
                        visualize=False)

[[{'bbox': [1985, 544, 228, 280],
   'confidence': 0.9461560249328613,
   'landmarks': [(2063, 640),
    (2167, 645),
    (2123, 702),
    (2061, 739),
    (2158, 742)]},
  {'bbox': [800, 726, 222, 274],
   'confidence': 0.9438166618347168,
   'landmarks': [(880, 840), (975, 826), (953, 887), (901, 939), (979, 927)]},
  {'bbox': [3345, 589, 217, 263],
   'confidence': 0.9390434622764587,
   'landmarks': [(3406, 697),
    (3508, 676),
    (3476, 736),
    (3433, 782),
    (3527, 763)]},
  {'bbox': [1490, 688, 184, 240],
   'confidence': 0.9253883361816406,
   'landmarks': [(1520, 783),
    (1603, 772),
    (1555, 826),
    (1540, 866),
    (1613, 855)]},
  {'bbox': [342, 791, 169, 198],
   'confidence': 0.9234484434127808,
   'landmarks': [(360, 864), (434, 854), (384, 895), (378, 936), (441, 927)]},
  {'bbox': [2829, 728, 231, 240],
   'confidence': 0.9144946932792664,
   'landmarks': [(2929, 802),
    (3017, 834),
    (2970, 872),
    (2897, 886),
    (2972, 914)]}],
 [{'bbox': [797, 

## **Gaussian Blur De-identification**

Apply Gaussian blur to detected faces for privacy protection. This method creates a smooth blur effect over face regions.

In [5]:
def GaussianBlur_deidentification(input_video):
    """Processing pipeline for GaussianBlur de-identification."""
    
    detector = FaceDetector(conf_threshold=0.5)
    
    # Create results folder if it doesn't exist
    results_folder = 'results'
    os.makedirs(results_folder, exist_ok=True)
    
    # Extract input filename and create output path in results folder
    input_filename = os.path.basename(input_video)
    base_name, ext = os.path.splitext(input_filename)
    output_video = os.path.join(results_folder, f'{base_name}_gaussianblur_deidentified{ext}')
    
    cap = cv.VideoCapture(input_video)
    fps = cap.get(cv.CAP_PROP_FPS)
    width = int(cap.get(cv.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv.CAP_PROP_FRAME_HEIGHT))
    
    # Setup output video writer
    fourcc = cv.VideoWriter_fourcc(*'mp4v')
    out = cv.VideoWriter(output_video, fourcc, fps, (width, height))
    
    frame_count = 0
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # Detect faces
        bboxes = detector.get_bboxes(frame)
        
        # Apply de-identification to each face
        for bbox in bboxes:
            x, y, w, h = bbox
            
            # Option 1: Blur
            face_region = frame[y:y+h, x:x+w]
            blurred = cv.GaussianBlur(face_region, (99, 99), 30)
            frame[y:y+h, x:x+w] = blurred
        
        # Write processed frame
        out.write(frame)
        frame_count += 1
        
        if frame_count % 30 == 0:
            print(f"Processed {frame_count} frames...")
    
    cap.release()
    out.release()
    print(f"De-identification complete! Output saved to {output_video}")

In [ ]:
video_path = r"video\sample_video_3.mp4"
GaussianBlur_deidentification(video_path)

## **Pixelation De-identification**

Apply pixelation effect to detected faces. This method reduces face resolution to create a mosaic/pixelated effect.

In [9]:
def Pixelated_deidentification(input_video):
    """Processing pipeline for Pixelated de-identification."""
    
    detector = FaceDetector(conf_threshold=0.5)
    
    # Create results folder if it doesn't exist
    results_folder = 'results'
    os.makedirs(results_folder, exist_ok=True)
    
    # Extract input filename and create output path in results folder
    input_filename = os.path.basename(input_video)
    base_name, ext = os.path.splitext(input_filename)
    output_video = os.path.join(results_folder, f'{base_name}_pixelated_deidentified{ext}')
    
    cap = cv.VideoCapture(input_video)
    fps = cap.get(cv.CAP_PROP_FPS)
    width = int(cap.get(cv.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv.CAP_PROP_FRAME_HEIGHT))
    
    # Setup output video writer
    fourcc = cv.VideoWriter_fourcc(*'mp4v')
    out = cv.VideoWriter(output_video, fourcc, fps, (width, height))
    
    frame_count = 0
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # Detect faces
        bboxes = detector.get_bboxes(frame)
        
        # Apply de-identification to each face
        for bbox in bboxes:
            x, y, w, h = bbox
            
            # Pixelate
            face_region = frame[y:y+h, x:x+w]
            # Use actual dimensions of the extracted region, not the original w, h
            actual_h, actual_w = face_region.shape[:2]
            if actual_h > 0 and actual_w > 0:  # Ensure valid region
                temp = cv.resize(face_region, (max(1, actual_w//10), max(1, actual_h//10)), interpolation=cv.INTER_LINEAR)

                pixelated = cv.resize(temp, (actual_w, actual_h), interpolation=cv.INTER_NEAREST)

                frame[y:y+h, x:x+w] = pixelated
        
        # Write processed frame
        out.write(frame)
        frame_count += 1
        
        if frame_count % 30 == 0:
            print(f"Processed {frame_count} frames...")
    
    cap.release()
    out.release()
    print(f"De-identification complete! Output saved to {output_video}")

In [ ]:
video_path = 'video/sample_video_3.mp4'
Pixelated_deidentification(video_path)

## **Black Box De-identification**

uses blackbox to cover up the face detected region.

In [11]:
def BlackBox_deidentification(input_video):
    """Processing pipeline for Black Box de-identification."""
    
    detector = FaceDetector(conf_threshold=0.5)
    
    # Create results folder if it doesn't exist
    results_folder = 'results'
    os.makedirs(results_folder, exist_ok=True)
    
    # Extract input filename and create output path in results folder
    input_filename = os.path.basename(input_video)
    base_name, ext = os.path.splitext(input_filename)
    output_video = os.path.join(results_folder, f'{base_name}_blackbox_deidentified{ext}')
    
    cap = cv.VideoCapture(input_video)
    fps = cap.get(cv.CAP_PROP_FPS)
    width = int(cap.get(cv.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv.CAP_PROP_FRAME_HEIGHT))
    
    # Setup output video writer
    fourcc = cv.VideoWriter_fourcc(*'mp4v')
    out = cv.VideoWriter(output_video, fourcc, fps, (width, height))
    
    frame_count = 0
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # Detect faces
        bboxes = detector.get_bboxes(frame)
        
        # Apply de-identification to each face
        for bbox in bboxes:
            x, y, w, h = bbox
            
            # Option 3: Black box (uncomment to use)
            cv.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 0), -1)
        
        # Write processed frame
        out.write(frame)
        frame_count += 1
        
        if frame_count % 30 == 0:
            print(f"Processed {frame_count} frames...")
    
    cap.release()
    out.release()
    print(f"De-identification complete! Output saved to {output_video}")

In [ ]:
video_path = 'video/sample_video_3.mp4'
BlackBox_deidentification(video_path)

---

## 👨‍💻 About Labellerr's Hands-On Learning in Computer Vision

Thank you for exploring this **Labellerr Hands-On Computer Vision Cookbook**! We hope this notebook helped you learn, prototype, and accelerate your vision projects.  
Labellerr provides ready-to-run Jupyter/Colab notebooks for the latest models and real-world use cases in computer vision, AI agents, and data annotation.

---
## 🧑‍🔬 Check Our Popular Youtube Videos

Whether you're a beginner or a practitioner, our hands-on training videos are perfect for learning custom model building, computer vision techniques, and applied AI:

- [How to Fine-Tune YOLO on Custom Dataset](https://www.youtube.com/watch?v=pBLWOe01QXU)  
  Step-by-step guide to fine-tuning YOLO for real-world use—environment setup, annotation, training, validation, and inference.
- [Build a Real-Time Intrusion Detection System with YOLO](https://www.youtube.com/watch?v=kwQeokYDVcE)  
  Create an AI-powered system to detect intruders in real time using YOLO and computer vision.
- [Finding Athlete Speed Using YOLO](https://www.youtube.com/watch?v=txW0CQe_pw0)  
  Estimate real-time speed of athletes for sports analytics.
- [Object Counting Using AI](https://www.youtube.com/watch?v=smsjBBQcIUQ)  
  Learn dataset curation, annotation, and training for robust object counting AI applications.
---

## 🎦 Popular Labellerr YouTube Videos

Level up your skills and see video walkthroughs of these tools and notebooks on the  
[Labellerr YouTube Channel](https://www.youtube.com/@Labellerr/videos):

- [How I Fixed My Biggest Annotation Nightmare with Labellerr](https://www.youtube.com/watch?v=hlcFdiuz_HI) – Solving complex annotation for ML engineers.
- [Explore Your Dataset with Labellerr's AI](https://www.youtube.com/watch?v=LdbRXYWVyN0) – Auto-tagging, object counting, image descriptions, and dataset exploration.
- [Boost AI Image Annotation 10X with Labellerr's CLIP Mode](https://www.youtube.com/watch?v=pY_o4EvYMz8) – Refine annotations with precision using CLIP mode.
- [Boost Data Annotation Accuracy and Efficiency with Active Learning](https://www.youtube.com/watch?v=lAYu-ewIhTE) – Speed up your annotation workflow using Active Learning.

> 👉 **Subscribe** for Labellerr's deep learning, annotation, and AI tutorials, or watch videos directly alongside notebooks!

---

## 🤝 Stay Connected

- **Website:** [https://www.labellerr.com/](https://www.labellerr.com/)
- **Blog:** [https://www.labellerr.com/blog/](https://www.labellerr.com/blog/)
- **GitHub:** [Labellerr/Hands-On-Learning-in-Computer-Vision](https://github.com/Labellerr/Hands-On-Learning-in-Computer-Vision)
- **LinkedIn:** [Labellerr](https://in.linkedin.com/company/labellerr)
- **Twitter/X:** [@Labellerr1](https://x.com/Labellerr1)

*Happy learning and building with Labellerr!*
